In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/u/shuhan/projects/vla')

from src.environments.highway_env.dataset import HighwayDataset, collate_fn

In [2]:
import torch
from torch.utils.data import DataLoader

In [5]:
%load_ext autoreload
%autoreload 2
from src.auto_labeling.highway_env.lane_change import LaneChangeTaskSpec
from src.models.vlas.cont_obs_token_action_cot_unified_token import ContObsTokenActionCOTVLAUnifiedToken

data_folder = '/storage/Datasets/highway_env/highway_fast_v0_dqn_meta_action_5_lanes/rollouts_train'
dataset = HighwayDataset(data_folder)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)


task_spec_func = LaneChangeTaskSpec

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import tqdm

cnt = 0
start_lane_id_to_goal = {id: [] for id in range(5)}

with tqdm.tqdm(total=10000) as pbar:
  for batch in dataloader:
    obs, act, valid_mask = batch
    cot_cfg = {'lanes_count': 5, 'max_hop': 4, 'cot_index_mode': 'both'}

    task_spec = task_spec_func(obs[0][valid_mask[0]].cpu().numpy(), act[0][valid_mask[0]].cpu().numpy(), cot_cfg)
    goal_spec = task_spec.get_goal_spec()
    hop_lane_ids, _ = task_spec._get_task_lane_ids()

    if len(hop_lane_ids) == 5:
      start_lane_id = hop_lane_ids[0]
      start_lane_id_to_goal[start_lane_id].append({'goal_spec': goal_spec, 'hop_lane_ids': hop_lane_ids})

      cnt += 1
      pbar.update(1)

    if cnt > 10000:
      break

  0%|          | 0/10000 [00:00<?, ?it/s]

10001it [00:14, 710.60it/s]                          


In [39]:
import pickle

save_path = '/u/shuhan/projects/vla/data/highway_env/lane_change_goal_spec_data.pkl'
with open(save_path, 'wb') as f:
  pickle.dump(start_lane_id_to_goal, f)